Imports

In [ ]:
import numpy as np
import openpyxl
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import scipy
from google.colab import files
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


from google.colab import drive
drive.mount('/content/drive')

Preprocessing

In [ ]:
path = "/Users/HajerSinan/Documents/Ph.D_Rutgers/Research/Olivia_Proteomics/FINAL FILES/BME_Clinical_Proteomics_Blanks_plus2310.csv"
raw_df = pd.read_csv(path, header=1)

df = raw_df.iloc[0:61, 1:]
print("Original shape:", df.shape)

# Drop columns with fewer than 5 non-zero values
cols_to_drop = []
for i in range(len(df.columns)):
    num_count = df.iloc[:, i].count()
    if num_count < 5:
        cols_to_drop.append(df.columns[i])
df = df.drop(cols_to_drop, axis=1)
print("After dropping sparse columns:", df.shape)

df = df.applymap(lambda x: pd.to_numeric(str(x).replace('*', ''), errors='coerce'))
missing_mask = df.isna()

# Impute in column batches
batch_size = 500
columns = df.columns
imputed_batches = []
mask_batches = []

for start in range(0, len(columns), batch_size):
    end = start + batch_size
    batch_cols = columns[start:end]
    batch_df = df[batch_cols]
    rf_estimator = RandomForestRegressor(n_estimators = 10, random_state=0)
    imputer = IterativeImputer(estimator=rf_estimator, max_iter=10)
    imputed_data = imputer.fit_transform(batch_df)
    imputed_batch = pd.DataFrame(imputed_data, columns=batch_cols)

    mask_batch = missing_mask[batch_cols]

    imputed_batches.append(imputed_batch)
    mask_batches.append(mask_batch)

df_imputed = pd.concat(imputed_batches, axis=1)
mask_combined = pd.concat(mask_batches, axis=1)
df_imputed = df_imputed[df.columns]
mask_combined = mask_combined[df.columns]

# Export to Excel and highlight imputed values
output_path = "/Users/HajerSinan/Documents/Python Workspace/imputed_data_highlighted_7.28.25.xlsx"
df_imputed_rounded = df_imputed.round(3)
df_imputed_rounded.to_excel(output_path, index=False)


## HIGHLIGHTING IMPUTATION

# Open with openpyxl to highlight
wb = openpyxl.load_workbook(output_path)
ws = wb.active

# Yellow fill for imputed values
fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

for row in range(mask_combined.shape[0]):
    for col in range(mask_combined.shape[1]):
        if mask_combined.iloc[row, col]:
            cell = ws.cell(row=row+2, column=col+1)  # +2 due to header and 1-indexing
            cell.fill = fill

wb.save(output_path)
print(f"Imputed Excel saved to: {output_path}")




Correlation

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename, engine="openpyxl")


spearman_corr, p_vals = scipy.stats.spearmanr(df)
p_x, p_y = p_vals.shape

# Thresholds
pval_threshold = 0.05
corr_very_strong = 0.7
corr_strong = 0.5
corr_moderate = 0.3

very_strong = PatternFill(start_color="00008B", end_color="00008B", fill_type="solid")  # Dark Blue
strong = PatternFill(start_color="4169E1", end_color="4169E1", fill_type="solid")        # Royal Blue
moderate = PatternFill(start_color="87CEFA", end_color="87CEFA", fill_type="solid")      # Light Sky Blue
weak = PatternFill(start_color="B0E0E6", end_color="B0E0E6", fill_type="solid")           # Powder Blue


valid_mask = ~((p_vals == 0) & (np.abs(spearman_corr) == 1))

very_strong_mask = valid_mask & (p_vals < pval_threshold) & (np.abs(spearman_corr) > corr_very_strong)
strong_mask = valid_mask & (p_vals < pval_threshold) & (np.abs(spearman_corr) > corr_strong) & (np.abs(spearman_corr) <= corr_very_strong)
moderate_mask = valid_mask & (p_vals < pval_threshold) & (np.abs(spearman_corr) > corr_moderate) & (np.abs(spearman_corr) <= corr_strong)
weak_mask = valid_mask & (p_vals < pval_threshold) & (np.abs(spearman_corr) <= corr_moderate)


headers = df.columns.tolist()

# Defining writer function
def format_and_save_excel(matrix, filename, apply_pvals=False, is_logp=False):
    wb = Workbook()
    ws = wb.active

    # Insert headers
    ws.cell(row=1, column=1).value = ""
    for j, header in enumerate(headers):
        ws.cell(row=1, column=j+2, value=header).font = Font(bold=True)
        ws.cell(row=j+2, column=1, value=header).font = Font(bold=True)

    # Insert values and apply formatting
    for i in range(p_y):
        for j in range(p_x):
            if apply_pvals:
                val = f"{spearman_corr[i, j]:.2f} ({p_vals[i, j]:.8f})"
            elif is_logp:
                val = -np.log10(p_vals[i, j]) if p_vals[i, j] > 0 else np.nan
            else:
                val = matrix[i, j]
            cell = ws.cell(row=i+2, column=j+2, value=val)

            # Apply highlighting
            if very_strong_mask[i, j]:
                cell.fill = very_strong
            elif strong_mask[i, j]:
                cell.fill = strong
            elif moderate_mask[i, j]:
                cell.fill = moderate
            elif weak_mask[i, j]:
                cell.fill = weak

    wb.save(filename)


correlation_matrix_7_28_25 = format_and_save_excel(None, "correlation_matrix_7.28.25.xlsx", apply_pvals=True)
correlation_r_values_7_28_25 = format_and_save_excel(spearman_corr, "correlation_r_values_7.28.25.xlsx")
correlation_p_values_7_28_25 = format_and_save_excel(p_vals, "correlation_p_values_7.28.25.xlsx")
correlation_neglogp_values_7_28_25 = format_and_save_excel(None, "correlation_neglogp_values_7.28.25.xlsx", is_logp=True)
